<a id='top'></a>

# Spreadsheet Transformations
1. [Transpose](#transpose)
2. [Common Samples](#common_samples)
3. [Merge](#merge)
4. [Select Genes](#select_genes)
5. [Cluster Averages](#cluster_averages)
6. [Select Categorical](#select_categorical)

In [1]:
# %%html
# <style>
# div.input {
#     display:none;
# }
# div.output_stderr{
#     display:none
# }
# </style>

In [2]:
#                                         single cell for select, open and transpose:
#                                         target directory set for docker run -v `pwd`:...   ==  mount user data
target_dir = '../data'

import warnings
warnings.filterwarnings('ignore')

import os
import sys

import pandas as pd
import numpy as np

from IPython.display import display, HTML, clear_output
import ipywidgets as widgets

from knpackage import toolbox as kn

#                                         Get list of (docker run -v) mounted files:
flist = os.listdir(target_dir)
FEXT = ['.tsv', '.txt', '.df']
my_file_list = []
for f in flist:
    if os.path.isfile(os.path.join(target_dir, f)):
        noNeed, f_ext = os.path.splitext(f)
        if f_ext in FEXT:
            my_file_list.append(f)

#                                         (docker run -v) mounted files was empty:
if len(my_file_list) <= 0:
    my_file_list.append('No Data')

In [3]:
print(my_file_list)

['gene_samples_small_labels.tsv', 'spreadsheet_One.txt', 'gene_samples_small.tsv', 'spreadsheet_Two.txt', 'transform_5_spreadsheet.tsv', 'gene_samples_1.tsv', 'gene_samples_1_list.txt', 'tcga_ucec_somatic_mutation_data.df', 'UCEC_phenotype.txt']


In [4]:
#                                         initialize test directory
test_dir = os.path.abspath('../test')
test_dir_listing = os.listdir(test_dir)
results_dir = os.path.join(test_dir, 'run_dir')

if os.path.isdir(results_dir) == False:
    os.mkdir(results_dir)
    
results_dir = os.path.join(results_dir, 'results')
if os.path.isdir(results_dir) == False:
    os.mkdir(results_dir)

output_dir = results_dir

### Note: 
#### View a file:
* **Select your spreadsheet file from the "Select File" dropdown listbox below.**
* **Press the "Visualize" button and the spreadsheet (or the upper-left part if it is too large) will be displayed.**

In [5]:
#                                         local function to open and Visualize:
def visualize_selected_file(button):
    """"""
    clear_output()
    try:
        if hasattr(button, 'fname_list'): 
            full_fname_list = button.fname_list
        else: 
            full_fname = os.path.join(target_dir, button.dropdown_box.value)
            full_fname_list = [full_fname]
        for full_fname in full_fname_list: 
            df = pd.read_csv(full_fname,sep='\t',header=0,index_col=0)
            Step = df.iloc[0:10,0:10];
            Step2 = HTML(Step.to_html())
            display(Step2)
            del Step2
        
    except OSError:
        print("No data to visualize! ")
        
def clear(change):
    clear_output()

<a id='transpose'></a>

<a id='transpose'></a>

## Transpose [[back to top]](#top) 

### Transpose a file:
* **Select your spreadsheet file from the "Select File" dropdown listbox below.**
* **Press the "Transpose" button and the transposed spreadsheet will be written to a file by the same name of the input file with "_T" appended in the same directory of the input file.**

In [6]:
transpose_file = my_file_list[1]
#                                         local function to open and transpose:
def transpose_selected_file(button):
    if len(my_file_list) == 0 or my_file_list[0] == 'No Data':
        return
    
    file_name = os.path.join(target_dir, flistbx.value)
    spreadsheet_df = pd.read_csv(file_name, sep='\t', index_col=0, header=0)
    spreadsheet_df = spreadsheet_df.transpose()
    name_base, file_extension = os.path.splitext(flistbx.value)
    outfile_name = os.path.join(output_dir, name_base + '_T.tsv')
    spreadsheet_df.to_csv(outfile_name, sep='\t')
    print('Output written to\n', outfile_name)
    button.fname_list = [outfile_name]
    visualize_selected_file(button)


#                                         Create and display the widget controls:
flistbx = widgets.Dropdown(
    options=my_file_list,
    value=transpose_file,
    description='Select File:'
)
display(flistbx)

visualize_file_button = widgets.Button(
    description='Visualize',
    disabled=False,
    button_style='',
    tooltip='file to visualiize',
    dropdown_box=flistbx
    )
visualize_file_button.on_click(visualize_selected_file)
display(visualize_file_button)


Widget Javascript not detected.  It may not be installed or enabled properly.


Widget Javascript not detected.  It may not be installed or enabled properly.


In [7]:

output_file_button = widgets.Button(
    description='Transpose',
    disabled=False,
    button_style='',
    tooltip='file to transpose',
    data_file_key='output_file_name'
    )
output_file_button.on_click(transpose_selected_file)
display(output_file_button)

Widget Javascript not detected.  It may not be installed or enabled properly.


<a id='common_samples'></a>

## Common Samples [[back to top]](#top)

### Return the subsets of two spreadsheets with only common samples of 2 spreadsheets: 
* **Select your two samples x phenotypes spreadsheet files respectively from the "Select File 1" and "Select File 2" dropdown listboxes below.**
* **Press the "Get Common Samples" button and the corresponding subsets of the spreadsheets with only common samples of 2 spreadsheets will be written to 2 files by the same names of 2 input files with "_Com" appended respectively in the same directory of the input files.**

In [8]:
#                                         local function to read files and get common samples and write:
common_samples_file_0 = my_file_list[1]
common_samples_file_1 = my_file_list[3]

def get_common_samples(button):
    if len(my_file_list) == 0 or my_file_list[0] == 'No Data':
        return
    
    file_name_1 = os.path.join(target_dir, flistbx_2.value)
    file_name_2 = os.path.join(target_dir, flistbx_3.value)
    sxp_1_df = pd.read_csv(file_name_1, sep='\t', index_col=0, header=0)
    sxp_2_df = pd.read_csv(file_name_2, sep='\t', index_col=0, header=0)
    sxp_1_gene_names = kn.extract_spreadsheet_gene_names(sxp_1_df)
    sxp_2_gene_names = kn.extract_spreadsheet_gene_names(sxp_2_df)
    common_samples_list = kn.find_common_node_names(sxp_1_gene_names, sxp_2_gene_names)
    sxp_1_trim_df,sxp_2_trim_df = sxp_1_df.loc[common_samples_list], sxp_2_df.loc[common_samples_list]
    name_base_1, file_extension_1 = os.path.splitext(flistbx_2.value)
    outfile_name_1 = os.path.join(output_dir, name_base_1 + '_Com.tsv')
    name_base_2, file_extension_2 = os.path.splitext(flistbx_3.value)
    outfile_name_2 = os.path.join(output_dir, name_base_2 + '_Com.tsv')
    sxp_1_trim_df.to_csv(outfile_name_1, sep='\t', index=True, header=True)
    sxp_2_trim_df.to_csv(outfile_name_2, sep='\t', index=True, header=True)
    print('Outputs written to\n', outfile_name_1,'\nand\n',outfile_name_2)
    button.fname_list = [outfile_name_1, outfile_name_2]
    visualize_selected_file(button)


#                                         Create and display the widget controls:
flistbx_2 = widgets.Dropdown(
    options=my_file_list,
    value=common_samples_file_0,
    description='Select File 1:'
)
display(flistbx_2)
flistbx_2.observe(clear, names='value')

visualize_file_button_2 = widgets.Button(
    description='Visualize',
    disabled=False,
    button_style='',
    tooltip='file to visualize',
    dropdown_box=flistbx_2
    )
display(visualize_file_button_2)
visualize_file_button_2.on_click(visualize_selected_file)

Widget Javascript not detected.  It may not be installed or enabled properly.


Widget Javascript not detected.  It may not be installed or enabled properly.


In [9]:
flistbx_3 = widgets.Dropdown(
    options=my_file_list,
    value=common_samples_file_1,
    description='Select File 2:'
)
display(flistbx_3)
flistbx_3.observe(clear, names='value')

visualize_file_button_3 = widgets.Button(
    description='Visualize',
    disabled=False,
    button_style='',
    tooltip='file to visualize',
    dropdown_box=flistbx_3
    )
display(visualize_file_button_3)
visualize_file_button_3.on_click(visualize_selected_file)


Widget Javascript not detected.  It may not be installed or enabled properly.


Widget Javascript not detected.  It may not be installed or enabled properly.


In [10]:

output_file_button_2 = widgets.Button(
    description='Get Common Samples',
    disabled=False,
    button_style='',
    tooltip='get common samples button',
    data_file_key='output_file_name'
    )
display(output_file_button_2)
output_file_button_2.on_click(get_common_samples)

flistbx_2.observe(clear, names='value')

Widget Javascript not detected.  It may not be installed or enabled properly.


<a id='merge'></a>

## Merge [[back to top]](#top)

### Combine two spreadsheets into one with all samples and phenotypes (NaN filled): 
* **Select your two samples x phenotypes spreadsheet files respectively from the "Select File 1" and "Select File 2" dropdown listboxes below.**
* **Press the "Merge" button and the union spreadsheet of those 2 spreadsheets will be written to a file by the name of first input file with the name of the second input file and "_Mrg" appended in the same directory of the input files.**

In [11]:
#                                         local function to read files and get common samples and write:
merge_file_0 = my_file_list[1]
merge_file_1 = my_file_list[3]

def merge(button):
    if len(my_file_list) == 0 or my_file_list[0] == 'No Data':
        return
    
    file_name_1 = os.path.join(target_dir, flistbx_4.value)
    file_name_2 = os.path.join(target_dir, flistbx_5.value)

    spreadsheet_1_df = pd.read_csv(file_name_1, sep='\t', index_col=0, header=0)
    spreadsheet_2_df = pd.read_csv(file_name_2, sep='\t', index_col=0, header=0)
    spreadsheet_1_samples = kn.extract_spreadsheet_gene_names(spreadsheet_1_df)
    spreadsheet_2_samples = kn.extract_spreadsheet_gene_names(spreadsheet_2_df)
    
    #all_samples_list = kn.find_unique_node_names(spreadsheet_1_samples, spreadsheet_2_samples)
    
    spreadsheet_1_phenotypes = list(spreadsheet_1_df.columns)
    spreadsheet_2_phenotypes = list(spreadsheet_2_df.columns)
    
    #all_phenotypes_list = kn.find_unique_node_names(spreadsheet_1_phenotypes, spreadsheet_2_phenotypes)
    
    spreadsheet_X_df = pd.concat([spreadsheet_1_df, spreadsheet_2_df], axis=1)
    name_base_1, file_extension_1 = os.path.splitext(flistbx_4.value)
    name_base_2, file_extension_2 = os.path.splitext(flistbx_5.value)
    # print(os.path.basename(name_base_2))
    # print(os.path.relpath(name_base_2,start=target_dir))
    outfile_name = os.path.join(output_dir, name_base_1 + '_' + name_base_2 + '_Mrg.tsv')
    spreadsheet_X_df.to_csv(outfile_name, sep='\t', index=True, header=True)
    print('Output written to\n', outfile_name)
    button.fname_list = [outfile_name]
    visualize_selected_file(button)

#                                         Create and display the widget controls:
flistbx_4 = widgets.Dropdown(
    options=my_file_list,
    value=merge_file_0,
    description='Select File 1:'
)
display(flistbx_4)

visualize_file_button_4 = widgets.Button(
    description='Visualize',
    disabled=False,
    button_style='',
    tooltip='file to visualize',
    dropdown_box=flistbx_4
    )
visualize_file_button_4.on_click(visualize_selected_file)
display(visualize_file_button_4)

Widget Javascript not detected.  It may not be installed or enabled properly.


Widget Javascript not detected.  It may not be installed or enabled properly.


In [12]:
flistbx_5 = widgets.Dropdown(
    options=my_file_list,
    value=merge_file_1,
    description='Select File 2:'
)
display(flistbx_5)

visualize_file_button_5 = widgets.Button(
    description='Visualize',
    disabled=False,
    button_style='',
    tooltip='file to visualize',
    dropdown_box=flistbx_5
    )
visualize_file_button_5.on_click(visualize_selected_file)
display(visualize_file_button_5)

Widget Javascript not detected.  It may not be installed or enabled properly.


Widget Javascript not detected.  It may not be installed or enabled properly.


In [13]:
output_file_button_3 = widgets.Button(
    description='Merge',
    disabled=False,
    button_style='',
    tooltip='merge button',
    data_file_key='output_file_name'
    )
output_file_button_3.on_click(merge)
display(output_file_button_3)

Widget Javascript not detected.  It may not be installed or enabled properly.


<a id='select_genes'></a>

## Select Genes [[back to top]](#top)

### Return one spreadsheet with only those genes selected from an input list: 
* **Select your genes x samples spreadsheet file and gene list file respectively from the "Select Spreadsheet File" and "Select Gene List File" dropdown listboxes below.**
* **Press the "Select Genes" button and the spreadsheet with only those genes selected will be written to a file by same name of the input spreadsheet file with "_Slct_Gn" appended in the same directory of the input files.**

In [14]:
# utility
select_genes_file_0 = my_file_list[5]
select_genes_file_1 = my_file_list[6]

def read_a_list_file(input_file_name):
    """
    Args:
        input_file_name:     full path name of a file containing a list
    Returns:
        a list that is contained in the file
    """
    with open(input_file_name, 'r') as fh:
        str_input = fh.read()
    return list(str_input.split())

#                                         local function to read files and get common samples and write:
def select_genes(button):
    if len(my_file_list) == 0 or my_file_list[0] == 'No Data':
        return
    
    file_name_1 = os.path.join(target_dir, flistbx_6.value)
    file_name_2 = os.path.join(target_dir, flistbx_7.value)

    gene_select_list = read_a_list_file(file_name_2)
    spreadsheet_df = pd.read_csv(file_name_1, sep='\t', index_col=0, header=0)
    gene_names = kn.extract_spreadsheet_gene_names(spreadsheet_df)
    intersection_names = sorted(kn.find_common_node_names(gene_names, gene_select_list))
    spreadsheet_intersected_df = spreadsheet_df.loc[intersection_names] 
    
    name_base_1, file_extension_1 = os.path.splitext(flistbx_6.value)
    # print(os.path.basename(name_base_2))
    # print(os.path.relpath(name_base_2,start=target_dir))
    outfile_name = os.path.join(output_dir, name_base_1 + '_Slt_Gn.tsv')
    spreadsheet_intersected_df.to_csv(outfile_name, sep='\t', index=True, header=True)
    print('Output written to\n', outfile_name)
    button.fname_list = [outfile_name]
    visualize_selected_file(button)


#                                         Create and display the widget controls:
flistbx_6 = widgets.Dropdown(
    options=my_file_list,
    value=select_genes_file_0,
    description='Select Spreadsheet File:'
)
display(flistbx_6)

visualize_file_button_6 = widgets.Button(
    description='Visualize',
    disabled=False,
    button_style='',
    tooltip='file to visualize',
    dropdown_box=flistbx_6
    )
visualize_file_button_6.on_click(visualize_selected_file)
display(visualize_file_button_6)

Widget Javascript not detected.  It may not be installed or enabled properly.


Widget Javascript not detected.  It may not be installed or enabled properly.


In [15]:
flistbx_7 = widgets.Dropdown(
    options=my_file_list,
    value=select_genes_file_1,
    description='Select Gene List File:'
)
display(flistbx_7)

visualize_file_button_7 = widgets.Button(
    description='Visualize',
    disabled=False,
    button_style='',
    tooltip='file to visualize',
    dropdown_box=flistbx_7
    )
visualize_file_button_7.on_click(visualize_selected_file)
display(visualize_file_button_7)

Widget Javascript not detected.  It may not be installed or enabled properly.


Widget Javascript not detected.  It may not be installed or enabled properly.


In [16]:
output_file_button_4 = widgets.Button(
    description='Select Genes',
    disabled=False,
    button_style='',
    tooltip='select genes button',
    data_file_key='output_file_name'
    )
output_file_button_4.on_click(select_genes)
display(output_file_button_4)

Widget Javascript not detected.  It may not be installed or enabled properly.


<a id='cluster_averages'></a>

##  Cluster Averages [[back to top]](#top)

### Return a spreadsheet of averages for each category given a genes x samples dataframe and a samples classification dictionary: 
* **Select your genes x samples spreadsheet file and samples classification dictionary file respectively from the "Select Spreadsheet File" and "Select Dictionary File" dropdown listboxes below.**
* **Press the "Get Cluster Averages" button the spreadsheet of averages for each category will be written to a file by the same name of the input spreadsheet file with "_Clst_Avg" appended in the same directory of the input files.**

In [17]:
#                                         local function to read files and get common samples and write:
cluster_averages_file_0 = my_file_list[2]
cluster_averages_file_1 = my_file_list[0]

def get_cluster_averages(button):
    if len(my_file_list) == 0 or my_file_list[0] == 'No Data':
        return
    file_name_1 = os.path.join(target_dir, flistbx_8.value)
    file_name_2 = os.path.join(target_dir, flistbx_9.value)

    spreadsheet_df = pd.read_csv(file_name_1, sep='\t', index_col=0, header=0)
    labels_df = pd.read_csv(file_name_2, sep='\t', index_col=0, names=['sample','cluster_number'])
    # print(spreadsheet_df)
    # print(labels_df)
    labels_dict = labels_df.to_dict()['cluster_number']
    cluster_numbers = list(np.unique(list(labels_dict.values())))
    labels = list(labels_dict.values())
    # labels == i is a boolean list
    cluster_ave_df = pd.DataFrame({i: spreadsheet_df.iloc[:, labels == i].mean(axis=1) for i in cluster_numbers})
    name_base_1, file_extension_1 = os.path.splitext(flistbx_8.value)
    # print(os.path.basename(name_base_2))
    # print(os.path.relpath(name_base_2,start=target_dir))
    outfile_name = os.path.join(output_dir, name_base_1 + '_Clst_Avg.tsv')
    cluster_ave_df.to_csv(outfile_name, sep='\t', index=True, header=True)
    print('Output written to\n', outfile_name)
    button.fname_list = [outfile_name]
    visualize_selected_file(button)

#                                         Create and display the widget controls:
flistbx_8 = widgets.Dropdown(
    options=my_file_list,
    value=cluster_averages_file_0,
    description='Select Spreadsheet File:'
)
display(flistbx_8)

visualize_file_button_8 = widgets.Button(
    description='Visualize',
    disabled=False,
    button_style='',
    tooltip='file to visualize',
    dropdown_box=flistbx_8
    )
visualize_file_button_8.on_click(visualize_selected_file)
display(visualize_file_button_8)


Widget Javascript not detected.  It may not be installed or enabled properly.


Widget Javascript not detected.  It may not be installed or enabled properly.


In [18]:
flistbx_9 = widgets.Dropdown(
    options=my_file_list,
    value=cluster_averages_file_1,
    description='Select Dictionary File:'
)
display(flistbx_9)

visualize_file_button_9 = widgets.Button(
    description='Visualize',
    disabled=False,
    button_style='',
    tooltip='file to visualize',
    dropdown_box=flistbx_9
    )
visualize_file_button_9.on_click(visualize_selected_file)
display(visualize_file_button_9)


Widget Javascript not detected.  It may not be installed or enabled properly.


Widget Javascript not detected.  It may not be installed or enabled properly.


In [19]:
output_file_button_5 = widgets.Button(
    description='Get Cluster Averages',
    disabled=False,
    button_style='',
    tooltip='get cluster averages button',
    data_file_key='output_file_name'
    )
output_file_button_5.on_click(get_cluster_averages)
display(output_file_button_5)

Widget Javascript not detected.  It may not be installed or enabled properly.


<a id='select_categorical'></a>

## Select Categorical [[back to top]](#top)

### From a genes x samples spreadsheet and a samples x phenotypes spreadsheet, return both spreadsheets with only the samples corresponding to a category in a phenotype: 
* **Select your genes x samples spreadsheet and samples x phenotypes spreadsheet respectively from the "Select G x S File" and "Select S x P File" dropdown listboxes below, and then select the phenotype id and select category from the next 2 dropdown listboxes.**
* **Press the "Select Categorical" button and the 2 spreadsheets with only the samples corresponding to a category in a phenotype will be written to 2 input files by the same names of those 2 files with "_Slct_Ctg" appended in the same directory of the input files.**

In [20]:
select_categorical_file_0 = my_file_list[4]
select_categorical_file_1 = my_file_list[3]
#                                         local function to read files and select categorical and write:
def select_categorical(button):
    if len(my_file_list) == 0 or my_file_list[0] == 'No Data':
        return
    
    file_name_1 = os.path.join(target_dir, flistbx_10.value)
    file_name_2 = os.path.join(target_dir, flistbx_11.value)
    spreadsheet_df = pd.read_csv(file_name_1, sep='\t', index_col=0, header=0)
    phenotype_df = pd.read_csv(file_name_2, sep='\t', index_col=0, header=0)
    phenotype_id = flistbx_12.value
    select_category = flistbx_13.value
    samples_list = phenotype_df.index[phenotype_df[phenotype_id] == select_category]
    #print(phenotype_df.index)
    # print(phenotype_df[phenotype_id][2])
    # print(samples_list)
    phenotype_category_df = phenotype_df.loc[samples_list]
    spreadsheet_category_df = spreadsheet_df[samples_list]
    
    name_base_1, file_extension_1 = os.path.splitext(flistbx_10.value)
    outfile_name_1 = os.path.join(output_dir, name_base_1 + '_Slct_Ctg.tsv')
    name_base_2, file_extension_2 = os.path.splitext(flistbx_11.value)
    outfile_name_2 = os.path.join(output_dir, name_base_2 + '_Slct_Ctg.tsv')
    spreadsheet_category_df.to_csv(outfile_name_1, sep='\t', index=True, header=True)
    phenotype_category_df.to_csv(outfile_name_2, sep='\t', index=True, header=True)
    print('Outputs written to\n', outfile_name_1,'\nand\n',outfile_name_2)
    button.fname_list = [outfile_name_1, outfile_name_2]
    visualize_selected_file(button)
    
def all_phenotypes(file_rel_path):
    """get all the phenotypes, i.e. column names, of the samples x phenotypes dataframe, 
 which is read from the file_rel_path file """
    try:
        phenotype_df = pd.read_csv(os.path.join(target_dir,file_rel_path), sep='\t', index_col=0, header=0)
        return list(phenotype_df.columns)
    except:
        return ['No Data or Invalid File']


def nan_unique(x):
    """a wrapper of the numpy.unique function that handles the NaN problem, 
    since numpy.unique will return multiple NaN's"""
    a = np.unique(x)
    r = []
    has_nan = False
    for i in a:
        if isinstance(i,float) and np.isnan(i):
            if has_nan: 
                continue
            else:
                has_nan = True
                r.append(i)
        else:
            r.append(i)
    return np.array(r)

def all_categories(file_rel_path,phenotype_id):
    """get all the categories, i.e. the values, of a specfic phenotype, 
    in the dataframe read from file_rel_path file"""
    try:
        phenotype_df = pd.read_csv(os.path.join(target_dir,file_rel_path), sep='\t', index_col=0, header=0)
        # print(list(np.unique(phenotype_df[phenotype_id])))
        return list(nan_unique(phenotype_df[phenotype_id]))
    except:
        return ['No Data or Invalid File']

#                                         Create and display the widget controls:
flistbx_10 = widgets.Dropdown(
    options=my_file_list,
    value=select_categorical_file_0,
    description='Select G x S File:'
)
display(flistbx_10)

visualize_file_button_10 = widgets.Button(
    description='Visualize',
    disabled=False,
    button_style='',
    tooltip='file to visulaize',
    dropdown_box=flistbx_10
    )
visualize_file_button_10.on_click(visualize_selected_file)
display(visualize_file_button_10)



Widget Javascript not detected.  It may not be installed or enabled properly.


Widget Javascript not detected.  It may not be installed or enabled properly.


In [21]:

flistbx_11 = widgets.Dropdown(
    options=my_file_list,
    value=select_categorical_file_1,
    description='Select S x P File:'
)
display(flistbx_11)

visualize_file_button_11 = widgets.Button(
    description='Visualize',
    disabled=False,
    button_style='',
    tooltip='file to visualize',
    dropdown_box=flistbx_11
    )
visualize_file_button_11.on_click(visualize_selected_file)
display(visualize_file_button_11)



Widget Javascript not detected.  It may not be installed or enabled properly.


Widget Javascript not detected.  It may not be installed or enabled properly.


In [22]:

flistbx_12 = widgets.Dropdown(
    options=all_phenotypes(select_categorical_file_1),
    value=all_phenotypes(select_categorical_file_1)[0],
    description='Select Phenotype Id:'
)

flistbx_13 = widgets.Dropdown(
    options=all_categories(select_categorical_file_1,flistbx_12.value),
    value=all_categories(select_categorical_file_1,flistbx_12.value)[0],
    description='Select Select Category:'
)

def handle_file_change(change):
    """the callback registered to handle changes in the 'value' 
    attribute of widget 'flist_11'"""
    flistbx_12.options = all_phenotypes(change['new']) 
    flistbx_12.value = all_phenotypes(change['new'])[0]
    flistbx_13.options = all_categories(change['new'],flistbx_12.value)
    flistbx_13.value = all_categories(change['new'],flistbx_12.value)[0]

flistbx_11.observe(handle_file_change, names='value')


def handle_phenotype_change(change):
    """the callback registered to handle changes in the 'value' 
    attribute of widget 'flist_12'"""
    flistbx_13.options = all_categories(flistbx_11.value,change['new'])
    flistbx_13.value = all_categories(flistbx_11.value,change['new'])[0]

flistbx_12.observe(handle_phenotype_change, names='value')

display(flistbx_12, flistbx_13)

output_file_button_6 = widgets.Button(
    description='Select Categorical',
    disabled=False,
    button_style='',
    tooltip='select categorical button',
    data_file_key='output_file_name'
    )
output_file_button_6.on_click(select_categorical)
display(output_file_button_6)

,TCGA-A5-A0G1,TCGA-A5-A0G3,TCGA-A5-A0G5,TCGA-A5-A0GA,TCGA-A5-A0GB,TCGA-A5-A0GD,TCGA-A5-A0GE,TCGA-A5-A0GG,TCGA-A5-A0GH,TCGA-A5-A0GI
GAUSPIRAT,0.948538,0.470411,0.834692,0.793820,0.115711,0.989160,0.407709,0.424900,0.735061,0.967961
ZQHAJJHBU,0.213151,0.115194,0.813162,0.807677,0.733912,0.153406,0.862237,0.144930,0.019860,0.196869
EMNQHKFHU,0.687267,0.219276,0.984807,0.245171,0.745360,0.356522,0.685051,0.032809,0.308030,0.172319
AIDBYQFWD,0.433509,0.401608,0.199667,0.408430,0.490161,0.089375,0.594615,0.629889,0.822173,0.534871
EZPPQKUNV,0.855013,0.015046,0.381002,0.573448,0.498565,0.507060,0.435569,0.662615,0.851215,0.563027
YOHRMXVIU,0.000573,0.592430,0.855953,0.837706,0.902555,0.409134,0.104318,0.494368,0.735257,0.570216
SBKIKCYOL,0.116484,0.166058,0.313235,0.470590,0.277679,0.380028,0.313996,0.625120,0.909162,0.370493
PUFCMBTVM,0.199653,0.115689,0.582334,0.138098,0.100535,0.133265,0.810141,0.677774,0.663989,0.954742
WAEJORIZO,0.272355,0.108076,0.024483,0.853082,0.248299,0.609909,0.181487,0.089960,0.839250,0.233763
IQHLAZRUW,0.915814,0.232222,0.711291,0.603413,0.901788,0.680728,0.849089,0.075202,0.864404,0.008695


,ICDO3site,stage,stage_simple,stage_ismeta,grade,grade_simple,residual
sample_id,,,,,,,
TCGA-A5-A0G1,c54.1,stage ia,i,0,grade 3,High Grade,r0
TCGA-A5-A0G3,c54.1,stage iiic2,iii,0,grade 3,High Grade,r0
TCGA-A5-A0G5,c54.1,stage ib,i,0,grade 3,High Grade,r0
TCGA-A5-A0GA,c54.1,stage iiic2,iii,0,grade 3,High Grade,r0
TCGA-A5-A0GB,c54.1,stage ib,i,0,grade 3,High Grade,r0
TCGA-A5-A0GD,c54.1,stage ia,i,0,grade 2,"Low grade (1,2)",r0
TCGA-A5-A0GE,c54.1,stage ia,i,0,grade 2,"Low grade (1,2)",r0
TCGA-A5-A0GG,c54.1,stage ia,i,0,grade 1,"Low grade (1,2)",r0
TCGA-A5-A0GH,c54.1,stage ia,i,0,grade 3,High Grade,r0
